In [1]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, accuracy_score, roc_auc_score
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import roc_auc_score, brier_score_loss
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import warnings
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MinMaxScaler  # StandardScalerからMinMaxScalerに変更
from sklearn.metrics import log_loss
import warnings
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss
from lightgbm import LGBMClassifier
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import optuna
from lightgbm import early_stopping, log_evaluation
warnings.filterwarnings('ignore')

In [2]:
# データの読み込み
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
# === データの前処理 ===

# Ageの処理
from kanjize import number2kanji, kanji2number
import re
import jaconv  # 日本語の全角・半角変換ライブラリ

def clean_age_extended(age):
    if pd.isnull(age):
        return None
    if '歳' in age:
        age = re.sub("歳", "", str(age))
    if '才' in age:
        age = re.sub("才", "", str(age))
    if '際' in age:
        age = re.sub("際", "", str(age))

    # 「10代」や「20代」の処理
    if "代" in age:
        try:
            base_age = int(re.sub("代", "", age))
            return base_age + 5  # 例えば「10代」は平均15歳とする
        except ValueError:
            return None
    # 半角数字の場合はそのまま数値に変換
    try:
        return int(age)
    except ValueError:
        # 漢数字をアラビア数字に変換
        return kanji2number(age)
# 例としてデータを変換
train['Age'] = train['Age'].apply(clean_age_extended)
test['Age'] = test['Age'].apply(clean_age_extended)

In [4]:
# TypeofContactの処理(ラベルエンコーディング)
train['TypeofContact'].fillna('Self Enquiry', inplace = True)
test['TypeofContact'].fillna('Self Enquiry', inplace = True)

In [5]:
# CityTierの処理
# そのまま使用

In [6]:
# DurationOfPitchの処理 
def clean_duration(duration):
    if pd.isna(duration):
        return np.nan
    duration_str = str(duration)
    if '分' in duration_str:
        num = float(''.join(filter(str.isdigit, duration_str)))
        return num * 60
    elif '秒' in duration_str:
        return float(''.join(filter(str.isdigit, duration_str)))
    return np.nan
train['DurationOfPitch'] = train['DurationOfPitch'].apply(clean_duration)
test['DurationOfPitch'] = test['DurationOfPitch'].apply(clean_duration)

In [7]:
# Occupation(ラベルエンコーディング)
# そのまま

In [8]:
# Gender(ラベルエンコーディング)
import re
import unicodedata
import pandas as pd

def normalize_category_gender(text):
    if not isinstance(text, str):
        return None
    # maleに関連する単語が含まれていればmale
    if any(word in text for word in ['Female', 'Ｆｅ\u3000Ｍａｌｅ','fe male', 'Ｆｅｍａｌｅ','ＦＥ\u3000ＭＡＬＥ', 'female', 'ｆｅ\u3000ｍａｌｅ', 'ｆｅｍａｌｅ', 'FEMALE', 'FE MALE','ＦＥＭＡＬＥ', 'Fe Male']):
        return 'female'
    elif any(word in text for word in ['Ｍａｌｅ','male', 'ｍａｌｅ','MALE','ＭＡＬＥ', 'Male'])and 'female' not in text:
        return 'male'
    return 'unknown'  # その他の場合

# DataFrameでの使用例
def normalize_product_column_gender(df, column_name):
    df = df.copy()
    df[column_name] = df[column_name].apply(normalize_category_gender)
    return df
train = normalize_product_column_gender(train, 'Gender')
test = normalize_product_column_gender(test, 'Gender')

In [9]:
# NumberOfPersonVisiting
# そのまま

In [10]:
# NumberOfPersonFollowups
outliers = [100, 200, 300, 400, 500, 600]
train['NumberOfFollowups'] = train['NumberOfFollowups'].replace(outliers, np.nan)
test['NumberOfFollowups'] = test['NumberOfFollowups'].replace(outliers, np.nan)

In [11]:
#ProductPitched(ラベルエンコーディング)
import re
import unicodedata
import pandas as pd

def normalize_category(text):
    """
    カテゴリー名を標準化する関数
    
    Parameters:
        text (str): 入力カテゴリー文字列
    
    Returns:
        str: 標準化されたカテゴリー名（'basic', 'standard', 'deluxe', 'super deluxe', 'king'のいずれか）
    """
    if not isinstance(text, str):
        return None
    
    # 前処理
    # 1. 小文字化
    # 2. 全角文字を半角に変換
    # 3. 特殊文字の正規化
    text = text.lower()
    text = unicodedata.normalize('NFKC', text)
    
    # 特殊文字や記号を通常のアルファベットに置換
    text = re.sub(r'[|×]', 'x', text)
    text = re.sub(r'[ıіℹ]', 'i', text)
    text = re.sub(r'[ꓢѕs]', 's', text)
    text = re.sub(r'[աꭰdԁ]', 'd', text)
    text = re.sub(r'[եℯeє]', 'e', text)
    text = re.sub(r'[ᗞ]', 'd', text)
    text = re.sub(r'[լl]', 'l', text)
    text = re.sub(r'[υu]', 'u', text)
    text = re.sub(r'[χx]', 'x', text)
    text = re.sub(r'[նn]', 'n', text)
    text = re.sub(r'[գg]', 'g', text)
    text = re.sub(r'[բb]', 'b', text)
    text = re.sub(r'[αа]', 'a', text)
    text = re.sub(r'[ϲс]', 'c', text)
    
    # スペースの正規化
    text = ' '.join(text.split())
    
    # カテゴリーの判定
    if any(word in text for word in ['basic', 'basix','basιc','baտic','basiς','βasic','basic','𐊡asic', 'вasic',]):
        return 'basic'
    elif any(word in text for word in ['super Deluxe','super dεluxe','super dexuxe''super deluxe']):
        return 'super deluxe'
    elif any(word in text for word in ['deluxε','dεluxε','deluxe', 'dexuxe', 'Deluxe'])and 'super deluxe' not in text:
        return 'deluxe'
    elif any(word in text for word in ['standard','standard','տtandard','staոdard']):
        return 'standard'
    elif any(word in text for word in ['kiոg','king','king']):
        return 'king'
    
    return text
# DataFrameでの使用例
def normalize_product_column(df, column_name):
    """
    DataFrameの特定のカラムのカテゴリーを標準化する関数
    
    Parameters:
        df (pandas.DataFrame): 入力データフレーム
        column_name (str): 標準化対象のカラム名
    
    Returns:
        pandas.DataFrame: カラムが標準化されたデータフレーム
    """
    df = df.copy()
    df[column_name] = df[column_name].apply(normalize_category)
    return df

# 使用例
train = normalize_product_column(train, 'ProductPitched')
test = normalize_product_column(test, 'ProductPitched')

In [12]:
#PreferredPropertyStar
#そのまま

In [13]:
#NumberOfTrips
def preprocess(value):
    # 値がfloat型の場合は文字列に変換
    if isinstance(value, float):
        value = str(value)
    # 数字のみの場合
    if re.match(r"^\d+$", value):
        return int(value)
    
    # "年に"が含まれる場合
    elif "年に" in value:
        times = int(re.search(r"(\d+)", value).group(1))
        return times 
    
    # "半年に"が含まれる場合
    elif "半年に" in value:
        times = int(re.search(r"(\d+)", value).group(1))
        return times * 2
    
    # その他の場合はNoneを返す
    else:
        return None

train['NumberOfTrips'] = train['NumberOfTrips'].apply(preprocess)
test['NumberOfTrips'] = test['NumberOfTrips'].apply(preprocess)

In [14]:
#Passport
#そのまま

In [15]:
#PitchSatisfactionScore
#そのまま

In [16]:
#Designation(ラベルエンコーディング)
#そのまま

In [17]:
#MonthlyIncome
def clean_income(income):
    if pd.isna(income):
        return np.nan
    income_str = str(income).replace(' ', '').replace(',', '')
    if '万円' in income_str:
        num = float(''.join(filter(str.isdigit, income_str)))
        return num * 1000
    return float(''.join(filter(str.isdigit, income_str))) / 10


train['MonthlyIncome'] = train['MonthlyIncome'].apply(clean_income)
test['MonthlyIncome'] = test['MonthlyIncome'].apply(clean_income)

In [18]:
# customer_infoの前処理
# 前処理関数
def preprocess(row):
    row = re.sub(r'[、,／／\t\n]', ' ', row)  # 区切り記号を統一
    row = re.sub(r'\s+', ' ', row).strip()  # 余分な空白を削除
    
    # 婚姻状況
    if re.search(r'(未婚)', row):
        marital_status = '未婚'
    elif re.search(r'(独身)',row):
        marital_status = '独身'
    elif re.search(r'(結婚済み)',row):
        marital_status = '結婚済み'
    elif re.search(r'(離婚済み)',row):
        marital_status = '離婚済み'
    else:
        marital_status = row
    
    # 車の所有状況
    if re.search(r'(車保有なし|車未所有|車未所持|車なし|乗用車なし)', row):
        car_status = '未所持'
    elif re.search(r'(車保有|車所持|自動車所有|車あり|自家用車あり)',row):
        car_status = '所持'
    else:
        car_status = row
    
    # 子供の人数
    child_match = re.search(r'(\d+)(人|児)', row)
    child_count = int(child_match.group(1)) if child_match else 0
    
    return [marital_status, car_status, child_count]

train[['married', 'car', 'child']] = train['customer_info'].apply(preprocess).apply(pd.Series)
test[['married', 'car', 'child']] = test['customer_info'].apply(preprocess).apply(pd.Series)

In [19]:
#欠損値を平均値で置き換える
numerical_columns = ['Age', 'DurationOfPitch', 'MonthlyIncome','NumberOfFollowups', 'NumberOfTrips']
#実験, 'NumberOfFollowups', 'NumberOfTrips'
for col in numerical_columns:
    median_value = train[col].median()
    train[col].fillna(median_value, inplace=True)
    test[col].fillna(median_value, inplace=True)

In [20]:
train['age_bin'] = pd.cut(train['Age'], bins=5, labels=False)
test['age_bin'] = pd.cut(test['Age'], bins=5, labels=False)
train['income_bin'] = pd.cut(train['MonthlyIncome'], bins=5, labels=False)
test['income_bin'] = pd.cut(test['MonthlyIncome'], bins=5, labels=False)

In [21]:
# モデルに使用する特徴量の選択
feature_columns = ['Age', 'CityTier', 'NumberOfPersonVisiting',
                  'NumberOfFollowups', 'PreferredPropertyStar', 'NumberOfTrips',
                  'DurationOfPitch', 'Passport', 'PitchSatisfactionScore', 'MonthlyIncome',
                  'TypeofContact', 'Occupation', 'Gender', 'ProductPitched', 'Designation',
                  'married', 'car', 'child','age_bin','income_bin']

In [22]:
X = train[feature_columns]
y = train['ProdTaken']

In [30]:
# 層化k-foldの初期化
skf = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)

In [31]:
# CatBoostのパラメータ設定
params_cat = {
    'iterations': 1000,
    'random_seed': 42,
    'eval_metric': 'AUC',
    'early_stopping_rounds': 50,
    'verbose': 100,
    'task_type': 'CPU',
    'bagging_temperature': 0, 
    'border_count': 128, 
    #4
    'depth': 1, 
    'iterations': 200, 
    'l2_leaf_reg': 3, 
    'learning_rate': 1, 
    #0.05
    'max_leaves': 31, 
    'min_data_in_leaf': 1, 
    'random_strength': 1
}

In [32]:
def predict_proba_cat(model, X_new):
    """
    CatBoostモデルを使用して新しいデータの予測確率を返す関数
    
    Parameters:
    -----------
    model : CatBoostClassifier
        学習済みのCatBoostモデル
    X_new : array-like
        予測したい新しいデータ
        
    Returns:
    --------
    array-like
        陽性クラスの予測確率
    """
    return model.predict_proba(X_new)[:, 1]

In [33]:
fold_auc_cat = []
fold_auc_cat_after = []
models_cat = []
models_cat_cali = []
# カテゴリカル変数のインデックスリスト（存在する場合）
cat_features = ['TypeofContact', 
                'Occupation', 
                'Gender',
                'Passport',
                'ProductPitched', 
                'Designation' , 
                'married',
                'car',
                'age_bin','income_bin',
                ]  # カテゴリカル変数のカラムインデックスを指定

In [34]:
# 交差検証
for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
    print(f"Fold {fold+1}")
    
    # データ分割
    X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
    y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
    
    # CatBoost用のデータセット作成
    train_pool = Pool(
        data=X_train,
        label=y_train,
        cat_features=cat_features
    )
    valid_pool = Pool(
        data=X_valid,
        label=y_valid,
        cat_features=cat_features
    )
    
    # モデルのインスタンス化と学習
    model = CatBoostClassifier(**params_cat)
    model.fit(
        train_pool,
        eval_set=valid_pool,
        use_best_model=True,
        plot=False
    )
    
    # 検証データでの確率予測
    y_pred_prob = predict_proba_cat(model, X_valid)

    # キャリブレーションの実行
    calibrated_model = CalibratedClassifierCV(
        model, 
        method='isotonic',  # または 'isotonic'
        cv='prefit'  # すでに学習済みのモデルを使用
        )
    
    calibrated_model.fit(X_train, y_train)
    pred_proba_calibrated = calibrated_model.predict_proba(X_valid)[:, 1]
    auc_after = roc_auc_score(y_valid, pred_proba_calibrated)
    
    # AUC計算
    auc = roc_auc_score(y_valid, y_pred_prob)
    print(f"Fold {fold+1} AUC: {auc_after}")
    fold_auc_cat.append(auc)
    fold_auc_cat_after.append(auc_after)
    models_cat.append(model)
    models_cat_cali.append(calibrated_model)

Fold 1
0:	test: 0.5058740	best: 0.5058740 (0)	total: 2.33ms	remaining: 465ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8496445966
bestIteration = 29

Shrink model to first 30 iterations.
Fold 1 AUC: 0.8518658681058313
Fold 2
0:	test: 0.5105798	best: 0.5105798 (0)	total: 1.64ms	remaining: 327ms
100:	test: 0.8307556	best: 0.8430960 (52)	total: 135ms	remaining: 132ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8430959589
bestIteration = 52

Shrink model to first 53 iterations.
Fold 2 AUC: 0.8451526918520469
Fold 3
0:	test: 0.5094116	best: 0.5094116 (0)	total: 1.25ms	remaining: 248ms
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8432275898
bestIteration = 13

Shrink model to first 14 iterations.
Fold 3 AUC: 0.8441325523232854
Fold 4
0:	test: 0.6986674	best: 0.6986674 (0)	total: 1.47ms	remaining: 293ms
100:	test: 0.8233005	best: 0.8352904 (54)	total: 146ms	remaining: 143ms
Stopped by overfitting detector  (50 iterations

In [35]:
all_auc = fold_auc_cat + fold_auc_cat_after
all_auc

[0.8496445965512702,
 0.8430959589311571,
 0.843227589838094,
 0.8352904311112577,
 0.836758254444701,
 0.9053666259854206,
 0.8247517894250749,
 0.8518658681058313,
 0.8451526918520469,
 0.8441325523232854,
 0.8336741762047697,
 0.835109014744203,
 0.9006827852360062,
 0.8237622456047763]

In [36]:
# 全foldの平均AUCを計算
mean_auc = np.mean(all_auc)
std_auc = np.std(all_auc)
print(f"\nMean AUC: {mean_auc:.4f} (±{std_auc:.4f})")
models_cat_cali_all = models_cat_cali + models_cat
models_cat_cali_all


Mean AUC: 0.8480 (±0.0238)


[CalibratedClassifierCV(cv='prefit',
                        estimator=<catboost.core.CatBoostClassifier object at 0x17a1cf470>,
                        method='isotonic'),
 CalibratedClassifierCV(cv='prefit',
                        estimator=<catboost.core.CatBoostClassifier object at 0x179a87e30>,
                        method='isotonic'),
 CalibratedClassifierCV(cv='prefit',
                        estimator=<catboost.core.CatBoostClassifier object at 0x171fc2540>,
                        method='isotonic'),
 CalibratedClassifierCV(cv='prefit',
                        estimator=<catboost.core.CatBoostClassifier object at 0x17a3ef740>,
                        method='isotonic'),
 CalibratedClassifierCV(cv='prefit',
                        estimator=<catboost.core.CatBoostClassifier object at 0x17a1cd430>,
                        method='isotonic'),
 CalibratedClassifierCV(cv='prefit',
                        estimator=<catboost.core.CatBoostClassifier object at 0x179d0b5f0>,
      

In [44]:
# テストデータでの予測
test_preds_cat_cali_all = np.mean([predict_proba_cat(model, test[feature_columns]) for model in models_cat_cali_all], axis=0)

In [45]:
# 提出ファイルの作成
submission = pd.DataFrame({
    'id': test['id'],
    'ProdTaken': test_preds_cat_cali_all
})
submission.to_csv('submission_catcali_all_7.csv', index=False)